Dla wybranych 200 próbek sygnału, zmniejsz częstotliwość próbkowania poprzez
pozostawienie co 2, co 5 oraz co 10 wartości (pamiętaj o metadanych). Sprawdź
jak subsampling wpływa na wymaganą długość próbki sygnału oraz na skuteczność uwierzytelniania.

# Imports

In [1]:
import json
import os

import numpy as np

from experiments.prepare_experiments import prepare_experiment_3
from scripts.voice_recognition_system import VoiceRecognitionSystem

C:\Virtual_Environments\venv_BIO_3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Define File Paths

In [2]:
parent_directory = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_directory = os.path.join(parent_directory, "data")

database_path = os.path.join(data_directory, "database")

incoming_authorized_users_path = os.path.join(
    data_directory, "database", "incoming_users", "authorized_users"
)
incoming_unauthorized_users_path = os.path.join(
    data_directory, "database", "incoming_users", "unauthorized_users"
)

In [3]:
experiment_name = "experiment_3"
experiment_path = os.path.join(data_directory, experiment_name)

# Initialize System

In [4]:
voice_recognition_system = VoiceRecognitionSystem(
    database_path, acceptance_threshold=0.7, allow_brute_force=False
)

Initializing database: 100%|██████████| 100/100 [00:19<00:00,  5.02it/s]


# Prepare Experiment

In [5]:
prepare_experiment_3(
    experiment_path,
    incoming_authorized_users_path,
    incoming_unauthorized_users_path,
    200,
)

# Conduct Experiment

In [6]:
for sample_rate_reduction in [0.1, 0.2, 0.5]:
    experiment_authorized_path = os.path.join(experiment_path, f"sample_rate_{sample_rate_reduction}", "authorized")
    experiment_unauthorized_path = os.path.join(experiment_path, f"sample_rate_{sample_rate_reduction}", "unauthorized")
    
    # Without brute force
    df_authorized_users = voice_recognition_system.verify_multiple_users(
        incoming_users_path=experiment_authorized_path,
    )
    df_unauthorized_users = voice_recognition_system.verify_multiple_users(
        incoming_users_path=experiment_unauthorized_path,
    )
    
    # With brute force
    voice_recognition_system.allow_brute_force = True

    df_unauthorized_users_brute_force = voice_recognition_system.verify_multiple_users(
        incoming_users_path=experiment_unauthorized_path
    )
    
    results_path = os.path.join(parent_directory, "results", experiment_name, f"sample_rate_{sample_rate_reduction}")

    if not os.path.exists(results_path):
        os.makedirs(results_path, exist_ok=True)
    
    # Calculating metrics
    configurations = {
        "default": (df_authorized_users, df_unauthorized_users),
        "brute_force": (
            df_authorized_users,
            df_unauthorized_users_brute_force,
        ),
    }
    for configuration_name in configurations.keys():
        df_authorized_users, df_unauthorized_users = configurations[configuration_name]
    
        false_acceptance_rate, false_rejection_rate = (
            voice_recognition_system.calculate_far_frr(
                df_authorized_users, df_unauthorized_users
            )
        )
        true_negative, false_positive, false_negative, true_positive = (
            voice_recognition_system.calculate_ROC_curve(
                df_authorized_users,
                df_unauthorized_users,
                roc_curve_path=os.path.join(
                    results_path, f"ROC_curve_{configuration_name}.png"
                ),
            )
        )
        accuracy = (true_positive + true_negative) / (
            true_positive + true_negative + false_positive + false_negative
        )
        access_granted_rate = voice_recognition_system.calculate_access_granted_rate(
            df_authorized_users
        )
        results_dict = {
            "FAR": np.round(false_acceptance_rate, 3),
            "FRR": np.round(false_rejection_rate, 3),
            "TN": true_negative,
            "FP": false_positive,
            "FN": false_negative,
            "TP": true_positive,
            "ACC": np.round(accuracy, 3),
            "AGR": np.round(access_granted_rate, 3),
        }
    
        # convert int64 values to native Python integers
        for key, value in results_dict.items():
            results_dict[key] = int(value) if isinstance(value, np.int64) else value
    
        with open(
            os.path.join(results_path, f"results_{configuration_name}.json"), "w"
        ) as json_file:
            json.dump(results_dict, json_file, indent=4)

Veryfing multiple users: 100%|██████████| 48/48 [00:07<00:00,  6.80it/s]
